In [1]:
import tensorflow as tf
import numpy as np

# 텐서보드를 이용하기 위한 코딩을 해보자. 우선 앞에것 그대로 읽는다.
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable = False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [2]:
# 신경망 계층마다 코드 덧붙이기
# with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표현
# 이름 붙이면 해당 이름의 변수가 어디에 쓰이는지 쉽게 확인가능
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    # tf.summary.scalar 를 이용해 수집하고 싶은 값들을 지정할 수 있습니다.
    tf.summary.scalar('cost', cost) # 손실값 추정을 위해 수집할 값 지정 코드, cost 텐서값 손쉽게 지정가능
    tf.summary.histogram('Weights', W1) # 가중치, 편향 등의 변화를 그래프로 보고 싶다면 추가

In [3]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [4]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    print('Step: %d, ' % sess.run(global_step),
         'Cost: %.3f' % sess.run(cost, feed_dict={X:x_data, Y: y_data}))
    summary = sess.run(merged, feed_dict={X: x_data, Y:y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

Step: 1,  Cost: 0.960
Step: 2,  Cost: 0.914
Step: 3,  Cost: 0.873
Step: 4,  Cost: 0.834
Step: 5,  Cost: 0.801
Step: 6,  Cost: 0.770
Step: 7,  Cost: 0.745
Step: 8,  Cost: 0.722
Step: 9,  Cost: 0.702
Step: 10,  Cost: 0.684
Step: 11,  Cost: 0.668
Step: 12,  Cost: 0.654
Step: 13,  Cost: 0.641
Step: 14,  Cost: 0.630
Step: 15,  Cost: 0.620
Step: 16,  Cost: 0.611
Step: 17,  Cost: 0.604
Step: 18,  Cost: 0.597
Step: 19,  Cost: 0.591
Step: 20,  Cost: 0.585
Step: 21,  Cost: 0.581
Step: 22,  Cost: 0.577
Step: 23,  Cost: 0.573
Step: 24,  Cost: 0.570
Step: 25,  Cost: 0.568
Step: 26,  Cost: 0.565
Step: 27,  Cost: 0.563
Step: 28,  Cost: 0.562
Step: 29,  Cost: 0.560
Step: 30,  Cost: 0.559
Step: 31,  Cost: 0.558
Step: 32,  Cost: 0.557
Step: 33,  Cost: 0.556
Step: 34,  Cost: 0.555
Step: 35,  Cost: 0.555
Step: 36,  Cost: 0.554
Step: 37,  Cost: 0.554
Step: 38,  Cost: 0.553
Step: 39,  Cost: 0.553
Step: 40,  Cost: 0.553
Step: 41,  Cost: 0.552
Step: 42,  Cost: 0.552
Step: 43,  Cost: 0.552
Step: 44,  Cost: 0.5

In [5]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)
prediction = tf.argmax(model, 1)
target = tf.argmax(Y,1)
print('예측값:', sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y:y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
